In [43]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [44]:
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers  import Dense,  GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

In [45]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [46]:
from tensorflow.keras.datasets import imdb

In [47]:
(x_train_text, y_train), (x_test_text, y_test) = imdb.load_data(num_words=10000)

In [48]:
print(f'Train-set size: {len(x_train_text)}, Test-set size: {len(x_test_text)}')

Train-set size: 25000, Test-set size: 25000


In [49]:
data_text = x_train_text  + x_test_text

In [50]:
x_train_text[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 2,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 2,
 349,
 2637,
 148,
 605,
 2,
 8003,
 15,
 123,
 125,
 68,
 2,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 2,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 2,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 2,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 154,
 462,
 33,
 89,
 78,
 2

In [51]:
y_train[1]

0

In [52]:
# We don't need but for example purpose

In [53]:
# tokenizer = Tokenizer(num_words=10000)
# tokenizer.fit_on_texts(data_text)

In [54]:
word_index = imdb.get_word_index()

In [55]:
word_index = {k: (v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

In [56]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [57]:
reverse_word_index[7]

'of'

In [58]:
def decode_review(text):
        return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [59]:
decode_review(x_train_text[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [60]:
y_train[0]

1

In [61]:
x_train_text[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [62]:
num_tokens = [len(tokens) for tokens in data_text]

In [63]:
num_tokens = np.array(num_tokens)

In [64]:
np.mean(num_tokens)

469.51784

In [65]:
np.max(num_tokens)

2697

In [66]:
max_length = np.mean(num_tokens) + 2 * np.std(num_tokens)

In [67]:
max_length = int(max_length)

In [68]:
max_length

960

In [69]:
np.sum(num_tokens < max_length) / len(num_tokens)

0.9474

In [70]:
x_train_pad = pad_sequences(x_train_text, maxlen=max_length, padding='pre', truncating='pre')
x_test_pad = pad_sequences(x_test_text, maxlen=max_length, padding='pre', truncating='pre')

In [71]:
x_train_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [72]:
decode_review(x_train_pad[0])

"<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA

In [73]:
model = Sequential()

In [74]:
embedding_size = 8

In [75]:
x_train_pad.shape

(25000, 960)

In [76]:
model.add(Embedding(input_dim=10000, output_dim=8, input_length=max_length, name="layer"))

In [77]:
model.add(GRU(16, return_sequences=True))

In [78]:
model.add(GRU(8, return_sequences=True))

In [79]:
model.add(GRU(4))

In [80]:
model.add(Dense(1, activation="sigmoid"))

In [81]:
optimizer = Adam(learning_rate=1e-3)
from tensorflow.keras import losses


In [82]:
model.compile(optimizer, loss=losses.binary_crossentropy, metrics=['accuracy'])

In [83]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer (Embedding)            (None, 960, 8)            80000     
_________________________________________________________________
unified_gru_3 (UnifiedGRU)   (None, 960, 16)           1248      
_________________________________________________________________
unified_gru_4 (UnifiedGRU)   (None, 960, 8)            624       
_________________________________________________________________
unified_gru_5 (UnifiedGRU)   (None, 4)                 168       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 82,045
Trainable params: 82,045
Non-trainable params: 0
_________________________________________________________________


In [84]:
model.fit(x_train_pad, y_train, validation_split=0.3, epochs=3, batch_size=64)

Train on 17500 samples, validate on 7500 samples
Epoch 1/3
17500/17500 [==============================] - 591s 34ms/sample - loss: 0.5426 - accuracy: 0.7034 - val_loss: 0.3752 - val_accuracy: 0.8407
Epoch 2/3
17500/17500 [==============================] - 580s 33ms/sample - loss: 0.2946 - accuracy: 0.8871 - val_loss: 0.3226 - val_accuracy: 0.8688
Epoch 3/3
17500/17500 [==============================] - 580s 33ms/sample - loss: 0.2162 - accuracy: 0.9249 - val_loss: 0.3562 - val_accuracy: 0.8485


In [85]:
model.evaluate(x_test_pad, y_test)

25000/25000 [==============================] - 158s 6ms/sample - loss: 0.3636 - accuracy: 0.8452


[0.36361657921791074, 0.8452]

In [ ]:
# Embedding

In [86]:
layer_embedding = model.get_layer('layer')

In [87]:
weights_embedding = layer_embedding.get_weights()[0]

In [88]:
weights_embedding.shape

(10000, 8)

In [96]:
token_good = word_index['good']
token_good

52

In [97]:
token_great = word_index['great']
token_great

87

In [100]:
token_bad = word_index['bad']
token_bad

78

In [101]:
token_horrible = word_index['horrible']
token_horrible

527

In [98]:
weights_embedding[token_good]

array([ 0.06714702,  0.01219818,  0.02102049, -0.05757077, -0.07319661,
       -0.0705094 ,  0.05807162, -0.05191341], dtype=float32)

In [99]:
weights_embedding[token_great]

array([ 0.0929797 ,  0.11203813,  0.08777402, -0.09231119, -0.16468936,
       -0.10216101,  0.10913248, -0.12123481], dtype=float32)

In [102]:
weights_embedding[token_bad]

array([-0.07129734, -0.11207188, -0.13952565,  0.12776974,  0.06507187,
        0.09111603, -0.12886843,  0.1182317 ], dtype=float32)

In [103]:
weights_embedding[token_horrible]

array([-0.09196126, -0.15159762, -0.1345791 ,  0.15743418,  0.15927073,
        0.10941967, -0.15325029,  0.1309389 ], dtype=float32)

In [118]:

inverse_map = dict(zip(word_index.values(), word_index.keys()))
def print_sorted_words(word, metric='cosine'):
    token = word_index[word]
    embedding = weights_embedding[token]
    distances = cdist(weights_embedding, [embedding], metric=metric).T[0]
    sorted_index = np.argsort(distances)
    sorted_distances =  distances[sorted_index]
    sorted_words = [inverse_map[token] for token in sorted_index if token !=0]
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print(f"{distance} - {word}")
    k = 10
    print(f'{word}')
    _print_words(sorted_words[0:k], sorted_distances[0:k])
    print("...")
    _print_words(sorted_words[-k:], sorted_distances[-k:])

In [122]:
print_sorted_words('horrible')

horrible
0.0 - horrible
0.008136842672485933 - incoherent
0.00905251916847416 - inane
0.009103009120110639 - exploits
0.009541590264147137 - unbelievably
0.00962376202943771 - obnoxious
0.009959120886829043 - terrible
0.01033704497344301 - mildly
0.011263286176972809 - nun
0.011357771752181334 - verdict
...
1.9885622796815219 - outstanding
1.9888244527926748 - extensive
1.9897036529017376 - favorite
1.9897595149054772 - fox
1.9897894741468964 - exceptional
1.9898905729988896 - chamberlain
1.9919662436701544 - abc
1.9930249499249442 - suspend
1.994285534971798 - sox
1.9961892256298155 - language
